In [1]:
import napari
from skimage import data
import numpy as np
from skimage import measure, morphology, filters
import pandas as pd

In [2]:
image = data.human_mitosis()
nrows, ncols = image.shape

In [3]:
image1 = image[:nrows//2, :ncols//2]
image2 = image[:nrows//2, ncols//2:]
image3 = image[nrows//2:, :ncols//2]
image4 = image[nrows//2:, ncols//2:]
image_list = [image1, image2, image3, image4]

In [4]:
labels_list = []
for image in image_list:
    filtered = filters.gaussian(image, 1)
    th = filters.threshold_otsu(filtered)
    binary = filtered > th
    labels = measure.label(binary)
    labels_list.append(labels)

In [5]:
tables_list = []
for labels, image, i in zip(labels_list, image_list, range(len(image_list))):
    table = pd.DataFrame(measure.regionprops_table(labels, image, properties=['label', 
                                                                                'area', 
                                                                                'axis_major_length', 
                                                                                'axis_major_length', 
                                                                                'feret_diameter_max', 
                                                                                'orientation', 
                                                                                'solidity', 
                                                                                'eccentricity', 
                                                                                'intensity_mean',
    ]))
    table['frame'] = i+1
    tables_list.append(table)

In [6]:
master_table = pd.concat(tables_list)

In [7]:
labels_stack = np.stack(labels_list)

In [8]:
labels_stack.shape

(4, 256, 256)

In [9]:
viewer = napari.Viewer()
viewer.add_labels(labels_stack, features=master_table)

<Labels layer 'labels_stack' at 0x254c1751e20>

In [10]:
master_table

,label,area,axis_major_length,feret_diameter_max,orientation,solidity,eccentricity,intensity_mean,frame
0,1,64,10.862780,10.770330,1.570796,1.000000,0.713074,50.250000,1
1,2,73,12.568236,13.000000,0.507038,0.935897,0.808680,59.054795,1
2,3,77,11.542712,11.704700,0.041654,0.987179,0.680056,52.558442,1
3,4,86,11.624222,12.083046,-1.391353,0.945055,0.570413,47.790698,1
4,5,137,13.928775,14.866069,1.338121,0.951389,0.435103,56.416058,1
...,...,...,...,...,...,...,...,...,...
124,125,94,12.501291,13.000000,1.429126,0.969072,0.633197,75.191489,4
125,126,66,11.105378,12.083046,1.361233,0.916667,0.699711,81.575758,4
126,127,46,12.025488,12.369317,-1.375409,0.920000,0.903905,73.630435,4
127,128,41,10.016467,10.198039,1.551212,0.953488,0.843950,57.243902,4
